In [1]:
#@title Run This Cell for Interactive Simulation

import ipywidgets as widgets
from ipywidgets import interactive, Layout, HTML, Label
from IPython.display import display
import matplotlib.pyplot as plt

# Constants (names are shorthand for probability of events they define)
initI = 5
initN = 1e6
initG_sn = 80
initG_sp = 99.9
initC_sn = 50
initC_sp = 70

layout = Layout(width='50%', height='30px')
style = {'description_width': 'initial'}

def slider(description, value, max=100):
  return widgets.FloatSlider(value=value, min=0, max=max, step=0.1, description=description, continuous_update=False, 
                               orientation='horizontal', readout=True, readout_format='.1f', layout=layout, style=style)

I_header = HTML(description='<b>Population</b>', style=style, layout=layout)

N_box = widgets.BoundedIntText(value=initN, min=0, max=1e20, step=1, description='Tested Population Size [𝑁]', continuous_update=False, disabled=False,
                               layout=layout, style=style)

I_slider = slider('Disease Rate [𝑃(𝐼)] (%)', initI, max=50)

G_header = HTML(description='<b>RT-PCR</b>', style=style, layout=layout)

G_sn_slider = slider('RT-PCR Sensitivity [𝑃(𝐺∣𝐼)] (%)', initG_sn)

G_sp_slider = slider('RT-PCR Specificity [𝑃(𝐺ᶜ∣𝐼ᶜ)] (%)', initG_sp)

C_header = HTML(description='<b>CT (Given Negative RT-PCR Result)</b>', style=style, layout=layout)

C_sn_slider = slider('CT Conditional Sensitivity [𝑃(𝐶∣𝐼∩𝐺ᶜ)] (%)', initC_sn)

C_sp_slider = slider('CT Conditional Specificity [𝑃(𝐶ᶜ∣𝐼ᶜ∩𝐺ᶜ)] (%)', initC_sp)

kwargs = {'I_header':I_header, 'N':N_box, 'I':I_slider, 'G_header':G_header, 'G_sn':G_sn_slider, 'G_sp':G_sp_slider, 'C_header':C_header, 'C_sn':C_sn_slider, 'C_sp':C_sp_slider}
def simulate(I_header, N, I, G_header, G_sn, G_sp, C_header, C_sn, C_sp):
  def show(text, value, extra=''):
    display(HTML('{} is <b>{}</b>{}'.format(text, value, extra)), layout=layout)
  
  def showInt(text, value, extra=''):
    show(text, round(value), extra)

  def showPerc(text, value):
    show(text, str(round(value*100, 2))+'%')

  I /= 100
  G_sn /= 100
  G_sp /= 100
  C_sn /= 100
  C_sp /= 100

  Ic = 1.0 - I
  p_g_i = G_sn * I
  p_g_ic = (1.0 - G_sp) * Ic
  p_gc_i = (1.0 - G_sn) * I
  p_gc_ic = G_sp * Ic
  p_c_gc_i = C_sn * p_gc_i
  p_c_gc_ic = (1.0 - C_sp) * p_gc_ic
  p_cc_gc_i = (1.0 - C_sn) * p_gc_i
  p_cc_gc_ic = C_sp * p_gc_ic


  display(widgets.HTML(description='<b>False Negative Analysis</b>', style=style, layout=layout))
  infected = I*N
  showInt('Total number of true COVID patients in the test population', infected)
  showInt('False negatives from RT-PCR (no CT)', p_gc_i*N)
  showInt("False positives by RT-PCR (no CT)", p_g_ic*N)
  tp = p_c_gc_i*N
  showInt("Benefit of CT: True positives on CT that would have been negative on RT-PCR only", tp, extra=' ({0:.2f}% of infected population)'.format(tp/infected*100))

  display(widgets.HTML(description='<b>Bayesian Analysis</b>', style=style, layout=layout))
  showPerc('Positive predictive value of RT-PCR', p_g_i / (p_g_i + p_g_ic))
  showPerc('Negative predictive value of RT-PCR', 1 - p_gc_i / (p_gc_i + p_gc_ic))
  showPerc('Overall accuracy of RT-PCR', (G_sn * I + G_sp * Ic))
  showPerc('Probability of infection given negative RT-PCR and positive CT', p_c_gc_i / (p_c_gc_i + p_c_gc_ic))
  showPerc('Probability of infection given negative RT-PCR and negative CT', p_cc_gc_i / (p_cc_gc_i + p_cc_gc_ic))

plot = interactive(simulate, {'manual':True}, layout=Layout(height='80px'), **kwargs)
output = plot.children[-1]
output.layout.height = '335px'
plot

interactive(children=(HTML(value='', description='<b>Population</b>', layout=Layout(height='30px', width='50%'…

### To study the issue of false negative RT-PCR tests probabilistically, we define 3 events and their complements:

> $G$ - the genetic (RT-PCR) test gives a positive result.  
  $G^C$ - the genetic (RT-PCR) test gives a negative result.

> $C$ - the CT test gives a positive result (for COVID-19).  
  $C^C$ - the CT test gives a negative result (for COVID-19).

> $I$ - patient is infected with COVID-19.  
  $I^C$ - patient is not infected with COVID-19.

### We also input the sensitivity and specificity of RT-PCR:

> $P(G \mid I)$ - RT-PCR sensitivity.  
  $P(G^C \mid I^C)$ - RT-PCR specificity.

### And the the sensitivity and specificity of CT, conditioned on a negative RT-PCR result:

> $P(C \mid G^C, I)$ - CT sensitivity given that RT-PCR is negative.  
  $P(C^C \mid G^C, I^C)$ - CT specificity given that RT-PCR is negative.

### Finally, we define the base disease rate in our experimental population:

> $P(I)$ - the probability that a random patient is infected with COVID-19.  
  $N$ - experimental population size

## False Negative Analysis
> Using the definition of conditional probability, $P(A \mid B) = \frac{P(A \cap B)}{P(B)}$, we calculate the false negative rate of an RT-PCR test as $P(G^C \cap I) = P(G^C \mid I)P(I)$, where $P(G^C \mid I) = 1 - P(G \mid I)$. We then simply multiply this by the population size $N$ to get the number of false negative RT-PCR cases.  

> Similarly, $P(G \cap I^C) = (1 - P(G^C \mid I^C))P(I^C)$, and multiplying this by $N$ gives the number of false positive RT-PCRs.  

> Factoring in CT conditioned on negative RT-PCR, we can calculate the conditional false positive rate as $P(C \cap (G^C,I^C)) = P(C \mid G^C, I^C)P(G^C \mid I^C)P(I^C)$, where $P(C \mid G^C, I^C) = 1 - P(C^C \mid G^C, I^C)$.

> Likewise, the conditional true positive rate is given as $P(C \cap (G^C,I)) = P(C \mid G^C, I)P(G^C \mid I)P(I)$, where $P(G^C \mid I) = 1 - P(G \mid I)$. These are the cases in which CT reveals an infected case where RT-PCR failed.

## Bayesian Analysis
> Using Bayes's Law, $P(A \mid B) = \frac{P(B \mid A)P(A)}{P(B)}$, and the law of total probability, $P(B) = P(B \cap A) + P(B \cap A^C)$, we can calculate the probability that a patient is infected with COVID-19 given a positive RT-PCR result as $P(I \mid G) = \frac{P(G \mid I)P(I)}{P(G)}$, where $P(G) = P(G \cap I) + P(G \cap I^C) = P(G \mid I)P(I) + (1 - P(G^C \mid I^C))P(I^C)$.

> Similarly, $P(I \mid G^C) = \frac{(1 - P(G \mid I))P(I)}{(1 - P(G \mid I))P(I) + P(G^C \mid I^C)P(I^C)}$ gives the probability that a patient is infected given negative RT-PCR.

> If RT-PCR is negative and we wish to factor in a follow-up positive CT result $C$, we can compute $P(I \mid G^C, C) = \frac{P(I \cap G^C \cap C)}{P(G^C \cap C)} = \frac{P(C,G^C,I)}{P(G^C,C,I) + P(G^C,C,I^C)} = \frac{P(C \mid G^C,I)P(G^C,I)}{P(C \mid G^C,I)P(G^C,I) + (1 - P(C^C \mid G^C,I^C))P(G^C,I^C)}$
  - $P(G^C,I) = (1 - P(G \mid I))P(I)$
  - $P(G^C,I^C) = P(G^C \mid I^C)P(I^C)$

> On the other hand, if our follow-up CT result is negative, we have $P(I \mid G^C, C^C) = \frac{P(C^C,G^C,I)}{P(G^C,C^C,I) + P(G^C,C^C,I^C)} = \frac{P(C^C,G^C,I)}{P(C^C,G^C,I) + P(C^C|G^C,I^C)P(G^C,I^C)}$
- $P(C^C,G^C,I) = (1 - P(C \mid G^C,I))P(G^C,I)$